In [1]:
from model.model import SegmentationModel
import torch

torch.manual_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = SegmentationModel()
model.to(device)

print(model)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

/home/vedpatel/.conda/envs/pyml/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
SegmentationModel(
  (model): CBAM_UNet(
    (enc1): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=3)
        (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.01, inplace=True)
        (3): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
        (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): LeakyReLU(negative_slope=0.01, inplace=True)
      )
      (1): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.01, inplace=True)
        (3): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
        (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): Le

In [ ]:
from dataset import COCODatasetLOADER
from pycocotools.coco import COCO
from config import imageDir, annFile, imageSize

coco = COCO(annFile)

dataset = COCODatasetLOADER(coco, imageDir, size=imageSize)

In [ ]:
img_to_tensor = dataset[2][0].unsqueeze(0).to(device)

In [ ]:
import torch
from torch.profiler import profile, record_function, ProfilerActivity

model = SegmentationModel().to(device)
model.eval()

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
             record_shapes=True) as prof:
    with record_function("model_inference"):
        with torch.no_grad():
            _ = model(img_to_tensor)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=20))


In [ ]:
torch.onnx.export(model, img_to_tensor, "Segmentation.onnx")
# View .onnx in Netron: https://netron.app

In [ ]:
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np


img_to_tensor = dataset[4][0].unsqueeze(0).to(device)

out = model(img_to_tensor)
out_mask = torch.sigmoid(out).squeeze().cpu().detach().numpy()

plt.figure(figsize=(6, 6))
plt.imshow(out_mask, cmap='gray')
plt.axis("off")
plt.show()